# Assignment 3: Extracting “Better‐Paper” Guidelines from OpenReview dataset

This notebook documents the steps, decisions, and code used to leverage local LLMs for mining common reviewer feedback and building a markdown of best practices.
In fact, since I don’t have deep expertise in ML, I leaned heavily on LLMs to do the heavy work of extracting tips from peer reviews. I used ollama first with DeepSeek-8B, but because of its poor answers, I switched to Qwen‑3-14B.
Here's an example of Qwen‑3-14B extracting some common concerns from the first three reviews of the dataset.

In [34]:
import ollama

question = r"""
I am analizing a dataset of OpenReview papers because I want to create a small markdown of general guidelines, to help authors writing better papers.
Here are three reviews of a paper I chose:
<|review1|>
 This paper proposes Recency Bias, an adaptive mini batch selection method for training deep neural networks. To select informative minibatches for training, the proposed method maintains a fixed size sliding window of past model predictions for each data sample. At a given iteration, samples which have highly inconsistent predictions within the sliding window are added to the minibatch. The main contribution of this paper is the introduction of sliding window to remember past model predictions, as an improvement over the SOTA approach: Active Bias, which maintains a growing window of model predictions. Empirical studies are performed to show the superiority of Recency Bias over two SOTA approaches. Results are shown on the task of (1) image classification from scratch and (2) image classification by fine-tuning pretrained networks. +ves: + The idea of using a sliding window over a growing window in active batch selection is interesting. + Overall, the paper is well written. In particular, the Related Work section has a nice flow and puts the proposed method into context. Despite the method having limited novelty (sliding window instead of a growing window), the method has been well motivated by pointing out the limitations in SOTA methods. + The results section is well structured. It*s nice to see hyperparameter tuning results; and loss convergence graphs in various learning settings for each dataset. Concerns: - The key concern about the paper is the lack of rigorous experimentation to study the usefulness of the proposed method. Despite the paper stating that there have been earlier work (Joseph et al, 2019 and Wang et al, 2019) that attempt mini-batch selection, the paper does not compare with them. This is limiting. Further, since the proposed method is not specific to the domain of images, evaluating it on tasks other than image classification, such as text classification for instance, would have helped validate its applicability across domains. - Considering the limited results, a deeper analysis of the proposed method would have been nice. The idea of a sliding window over a growing window is a generic one, and there have been many efforts to theoretically analyze active learning over the last two decades. How does the proposed method fit in there? (For e.g., how does the expected model variance change in this setting?) Some form of theoretical/analytical reasoning behind the effectiveness of recency bias (which is missing) would provide greater insights to the community and facilitate further research in this direction. - The claim of 20.5% reduction in test error mentioned in the abstract has not been clearly addressed and pointed out in the results section of the paper. - On the same note, the results are not conclusively in favor of the proposed method, and only is marginally better than the competitors. Why does online batch perform consistently than the proposed method? There is no discussion of these inferences from the results. - The results would have been more complete if results were shown in a setting where just recency bias is used without the use of the selection pressure parameter. In other words, an ablation study on the effect of the selection pressure parameter would have been very useful. - How important is the warm-up phase to the proposed method? Considering the paper states that this is required to get good estimates of the quantization index of the samples, some ablation studies on reducing/increasing the warm-up phase and showing the results would have been useful to understand this. - Fig 4: Why are there sharp dips periodically in all the graphs? What do these correspond to? - The intuition behind the method is described well, however, the proposed method would have been really solidified if it were analysed in the context of a simple machine learning problem (such as logistic regression). As an example, verifying if the chosen minibatch samples are actually close to the decision boundary of a model (even if the model is very simple) would have helped analyze the proposed method well. Minor comments: * It would have been nice to see the relation between the effect of using recency bias and the difficulty of the task/dataset. * In the 2nd line in Introduction, it should be *deep networks* instead of *deep networks netowrks*. * Since both tasks in the experiments are about image classification, it would be a little misleading to present them as *image classification* and *finetuning*. A more informative way of titling them would be *image classification from scratch* and *image classification by finetuning*. * In Section 3.1, in the LHS of equation 3, it would be appropriate to use P(y_i/x_i; q) instead of P(y/x_i; q) since the former term was used in the paragraph. =====POST-REBUTTAL COMMENTS======== I thank the authors for the response and the efforts in the updated draft. Some of my queries were clarified. However, unfortunately, I still think more needs to be done to explain the consistency of the results and to study the generalizability of this work across datasets. I retain my original decision for these reasons.
<|review1|>
<|review2|>
This paper proposes an interesting heuristic of batch construction from samples. Instead of the usual random sampling, the authors to sample based on some measures of the ``uncertainty”. To be specific, the uncertainty is measured as a normalized entropy estimated from a window of historical predictions. I like the idea of designing more sophisticated ways to encourage more exploration over the samples that the model is not good at. The thought is similar as active learning. It is interesting to see how similar thought can be used to improve the performance of the algorithm in the general batch gradient descent setting. On the other hand I am not quite convinced the proposed way is truly better. The main concern is the experiments do not quite show the state-of-the-art result at all. It is not even close on MNIST, CIFAR-10 and CIFAR-100. Also those datasets are relatively small one. Can authors add results on larger datasets such as tiny image net? Besides this main concern I also have some worries about the design of the algorithm. I listed them below: 1. The vanilla stochastic gradient descent can be roughly justified since the expectation of the stochastic gradient is the true gradient of the loss. Now with the proposed heuristic will this still be true? 2. Is there any guarantee the algorithm can converge? It is not clear to me as the optimization proceeds the ``uncertainty” may oscillate. Is there any condition when the convergence is guaranteed? 3. As the number of classes grows the estimation of the entropy itself is a tough problem. Is there any way to mitigate this issue other than increase the window size? Another minor comment: Could the authors add more explanation on equation (4)? For example, is related to the maximum entropy led by a uniform distribution, and the summation term in (4) is related to the empirical entropy.
<|review2|>
<|review3|>
This paper explores a well motivated but very heuristic idea for selecting the next samples to train on for training deep learning models. This method relies on looking at the uncertainty of predictions of in the recent history of statements and preferring those instances that have a predictive uncertainty over the recent predictions. This allows the training method to train on instances that are neither too hard nor too easy and focus on reducing the uncertainty whenever it has the greatest potential gain to do so. There are two extra components that make this method work: - Windowing: only looking at the recent history of the instances which has two effects: firstly, the current state of the model is explored which gives a more recent assessment relative to the current state of the model. Secondly, it makes the algorithm faster by reducing the overhead of analyzing the prediction history of samples. - Annealing the selection bias: as the training goes on the selection becomes more random and less biased. This approach is evaluated in on three simple data-sets: MNIST, CIFAR-10 and CIFAR-100. Although this is a very limited subset of models, the results are consistent and statistically significant, although their effect is not really huge. The paper gives very little theoretical justification or analysis of the results but gives only the presented empirical evidence which seems to support the hypothesis on the efficacy of the approach. Another drawback of the approach is that it introduces new hyperparameters: those governing the annealing schedule for the selection bias. Since the approach seems efficient in a relatively constrained setup, it can be reasonably expected that it might be helpful in more general situations, therefore. On the other hand, since it is only evaluated on three very similar tasks, it limits the conclusiveness of the results. That*s why I would for weak accept. In the presence of more empirical (or even theoretical) evidence, I would vote for strong accept.
<|review3|>
Can you help me creating the markdown guidelines, inserting common mistakes that are worth citing and the best solution to fix them?
I want something like:
<|result|>
# 📝 Paper Writing Guidelines for Authors

This guide outlines key principles to ...

## 🧪 Rigorous Experimentation

Many submissions fall short on experimental rigor—insufficient controls, lack of repeatability, vague protocol descriptions, or no statistical power analysis.

**Best-Practice Solutions:**

1. **Define Hypotheses & Variables**
    * State null and alternative hypotheses explicitly.

...

## 2. 📊 Comparison to Prior Work
<|result|>

Answer me with only the final guideline markdown.
"""


response = ollama.chat(model='qwen3:14b', messages = [
    {
        "role": "user",
        "content": question
    }
])

response = response['message']['content']
print(response)

<think>
Okay, I need to help the user create markdown guidelines for authors based on the reviews they provided. Let me start by analyzing the three reviews to identify common issues and areas for improvement.

First, Review 1 points out that the paper lacks rigorous experimentation. They didn't compare with earlier works like Joseph et al. and Wang et al., and didn't test on other domains like text classification. Also, there's a lack of theoretical analysis and some results are not conclusive. The abstract's claim about a 20.5% reduction in test error isn't addressed properly. They also mention issues with ablation studies and the need for more analysis on the warm-up phase and the selection pressure parameter.

Review 2 mentions that the experiments don't show state-of-the-art results on standard datasets like MNIST, CIFAR-10, and CIFAR-100. The author is concerned about the method's effectiveness and suggests testing on larger datasets like Tiny ImageNet. There are also questions a

## 💡 The initial idea
At this point, the initial idea was to go through the reviews in batches of three (since each paper has three reviews), extract the main criticisms, and add them to a text file (already filled with some tips to guide the LLM) collecting all the feedback gathered so far. To do this, I would prompt the LLM to expand the markdown document if it identified any new suggestions. I thought also to maintain a list of best solutions that could be adopted to fix the concerns.

Here's the code:

In [ ]:
import ollama
import openpyxl
from io import SEEK_END

# this file contains the template result we want to get from the model
starting_result = open("starting_result.txt", "r")
final_result = open("final_result.txt", "w+")
final_result.write(starting_result.read())
final_result.seek(0, SEEK_END)
starting_result.close()

def get_main_question(result, review1, review2, review3):
    return f"""I am analizing a dataset of OpenReview papers because I want to create a small markdown of general guidelines, to help authors writing better papers.
So I collected some reviews and this is the result I created:
<|result|>
{result}
<|result|>
Now I have three reviews of a paper I chose.
I want you to find any new common mistakes that are worth citing and add them as a new heading to the markdown I provided you. Mantain the original style of the markdown (with headings, emoji in the headings, no headings of third level, etc.). If the mistakes are already present, do not add them. If you don't find any new mistake, leave the markdown as before.
Here are the reviews:
<|review1|>
{review1}
<|review1|>
<|review2|>
{review2}
<|review2|>
<|review3|>
{review3}
<|review3|>
Answer me with only the final guideline markdown.
"""

sheet = openpyxl.load_workbook("or_2020.xlsx").active

for row in range(5, 40, 3): # there should be sheet.max_row, but with my pc would take ~80 hours
    review1 = sheet.cell(row, 6).value
    review2 = sheet.cell(row+1, 6).value
    review3 = sheet.cell(row+2, 6).value
    final_question = get_main_question(final_result.read(), review1, review2, review3)
    final_result.seek(0, SEEK_END)
    response = ollama.chat(model='qwen3:14b', messages = [
        {
            "role": "user",
            "content": final_question
        }
    ])
    response = response['message']['content']
    if response.find('```markdown') != -1:
        response = response.split('```markdown')[1].split('```')[0] # get the guideline markdown part
    elif response.find('#') != -1:
        response = response.split('#')[1]
    final_result.write(response)

final_result.seek(0)
uniform_question = f"""
Uniform this markdown to have a uniform style (for example every heading have an emoji, no subheading of third level, start with the title "# 📝 Paper Writing Guidelines for Authors", etc). Finally, if there are headings that are too similar uniform them in a single heading.
Here's the markdown:
<|markdown|>
{final_result.read()}
<|markdown|>
"""
response = ollama.chat(model='qwen3:14b', messages = [
    {
        "role": "user",
        "content": uniform_question
    }
])
response = response['message']['content']
final_result.seek(0, SEEK_END)
final_result.write(response)
print(response)
final_result.close()

<think>
Okay, the user wants me to uniformize their markdown. Let me start by understanding the requirements.

First, they mentioned that every heading should have an emoji. The original markdown already has emojis in the main headings like 🧪, 📊, etc. But I need to check if all headings have emojis. The main title is "# 📝 Paper Writing Guidelines for Authors" which has an emoji, and the subheadings like "1. 🧪 Rigorous Experimentation" also have emojis. So that's covered.

Next, they want no subheadings of the third level. Looking at the original markdown, the structure is main headings (##) and then numbered lists with subpoints. There are no third-level headings (###), so that's already in place. But I should confirm that the user doesn't want any, so maybe just ensure that the existing structure doesn't have any.

They want to start with the title "# 📝 Paper Writing Guidelines for Authors". The original markdown does that, so that's good.

Then, they mentioned if there are headings t

The result turned out to be not too bad, but some problems arised:

- **Prompt‐length explosion:** as the markdown grows, the LLM allucinates or cuts off.
- **Inconsistency:** later generations sometimes rephrase or lose earlier entries.
- **Throughput:** re‑sending the entire file for each triple becomes slow.

For this reason, I decided to revise the strategy.

## 🔄 Revised Strategy: Separate Extraction & Clustering

Rather than carrying forward the cumulative output and feeding it back to the LLM each time, I decided to extract the common concerns from each triplet of reviews and save them to a file. Once all reviews are processed, I generate a neatly formatted Markdown document that lists each "cluster" of concerns alongside recommended best practices for addressing them. I also discovered that Ollama lets you include a chat history, so that each time I ask a question I can attach an example of extracting common guidelines, providing a bit of context.

Here's the code:

In [ ]:
import ollama
import openpyxl
from io import SEEK_END

final_result = open("final_result.txt", "w+")

def get_main_question(review1, review2, review3):
    return f"""Perfect, now I have other three reviews of a paper I chose:
<|review1|>
{review1}
<|review1|>
<|review2|>
{review2}
<|review2|>
<|review3|>
{review3}
<|review3|>
Answer me with only the common concern that the reviewers have about these three reviews, to help authors know in advance what are the common concerns. Answer with only the bullet points without any other explanation.
"""

question = r"""
I am analizing a dataset of OpenReview papers with reviews of the papers because I want to extract some common concern that the reviewers have, to help authors know in advance what are the common concerns.
In particular, I have three reviews of a paper I chose:
<|review1|>
 This paper proposes Recency Bias, an adaptive mini batch selection method for training deep neural networks. To select informative minibatches for training, the proposed method maintains a fixed size sliding window of past model predictions for each data sample. At a given iteration, samples which have highly inconsistent predictions within the sliding window are added to the minibatch. The main contribution of this paper is the introduction of sliding window to remember past model predictions, as an improvement over the SOTA approach: Active Bias, which maintains a growing window of model predictions. Empirical studies are performed to show the superiority of Recency Bias over two SOTA approaches. Results are shown on the task of (1) image classification from scratch and (2) image classification by fine-tuning pretrained networks. +ves: + The idea of using a sliding window over a growing window in active batch selection is interesting. + Overall, the paper is well written. In particular, the Related Work section has a nice flow and puts the proposed method into context. Despite the method having limited novelty (sliding window instead of a growing window), the method has been well motivated by pointing out the limitations in SOTA methods. + The results section is well structured. It*s nice to see hyperparameter tuning results; and loss convergence graphs in various learning settings for each dataset. Concerns: - The key concern about the paper is the lack of rigorous experimentation to study the usefulness of the proposed method. Despite the paper stating that there have been earlier work (Joseph et al, 2019 and Wang et al, 2019) that attempt mini-batch selection, the paper does not compare with them. This is limiting. Further, since the proposed method is not specific to the domain of images, evaluating it on tasks other than image classification, such as text classification for instance, would have helped validate its applicability across domains. - Considering the limited results, a deeper analysis of the proposed method would have been nice. The idea of a sliding window over a growing window is a generic one, and there have been many efforts to theoretically analyze active learning over the last two decades. How does the proposed method fit in there? (For e.g., how does the expected model variance change in this setting?) Some form of theoretical/analytical reasoning behind the effectiveness of recency bias (which is missing) would provide greater insights to the community and facilitate further research in this direction. - The claim of 20.5% reduction in test error mentioned in the abstract has not been clearly addressed and pointed out in the results section of the paper. - On the same note, the results are not conclusively in favor of the proposed method, and only is marginally better than the competitors. Why does online batch perform consistently than the proposed method? There is no discussion of these inferences from the results. - The results would have been more complete if results were shown in a setting where just recency bias is used without the use of the selection pressure parameter. In other words, an ablation study on the effect of the selection pressure parameter would have been very useful. - How important is the warm-up phase to the proposed method? Considering the paper states that this is required to get good estimates of the quantization index of the samples, some ablation studies on reducing/increasing the warm-up phase and showing the results would have been useful to understand this. - Fig 4: Why are there sharp dips periodically in all the graphs? What do these correspond to? - The intuition behind the method is described well, however, the proposed method would have been really solidified if it were analysed in the context of a simple machine learning problem (such as logistic regression). As an example, verifying if the chosen minibatch samples are actually close to the decision boundary of a model (even if the model is very simple) would have helped analyze the proposed method well. Minor comments: * It would have been nice to see the relation between the effect of using recency bias and the difficulty of the task/dataset. * In the 2nd line in Introduction, it should be *deep networks* instead of *deep networks netowrks*. * Since both tasks in the experiments are about image classification, it would be a little misleading to present them as *image classification* and *finetuning*. A more informative way of titling them would be *image classification from scratch* and *image classification by finetuning*. * In Section 3.1, in the LHS of equation 3, it would be appropriate to use P(y_i/x_i; q) instead of P(y/x_i; q) since the former term was used in the paragraph. =====POST-REBUTTAL COMMENTS======== I thank the authors for the response and the efforts in the updated draft. Some of my queries were clarified. However, unfortunately, I still think more needs to be done to explain the consistency of the results and to study the generalizability of this work across datasets. I retain my original decision for these reasons.
<|review1|>
<|review2|>
This paper proposes an interesting heuristic of batch construction from samples. Instead of the usual random sampling, the authors to sample based on some measures of the ``uncertainty”. To be specific, the uncertainty is measured as a normalized entropy estimated from a window of historical predictions. I like the idea of designing more sophisticated ways to encourage more exploration over the samples that the model is not good at. The thought is similar as active learning. It is interesting to see how similar thought can be used to improve the performance of the algorithm in the general batch gradient descent setting. On the other hand I am not quite convinced the proposed way is truly better. The main concern is the experiments do not quite show the state-of-the-art result at all. It is not even close on MNIST, CIFAR-10 and CIFAR-100. Also those datasets are relatively small one. Can authors add results on larger datasets such as tiny image net? Besides this main concern I also have some worries about the design of the algorithm. I listed them below: 1. The vanilla stochastic gradient descent can be roughly justified since the expectation of the stochastic gradient is the true gradient of the loss. Now with the proposed heuristic will this still be true? 2. Is there any guarantee the algorithm can converge? It is not clear to me as the optimization proceeds the ``uncertainty” may oscillate. Is there any condition when the convergence is guaranteed? 3. As the number of classes grows the estimation of the entropy itself is a tough problem. Is there any way to mitigate this issue other than increase the window size? Another minor comment: Could the authors add more explanation on equation (4)? For example, is related to the maximum entropy led by a uniform distribution, and the summation term in (4) is related to the empirical entropy.
<|review2|>
<|review3|>
This paper explores a well motivated but very heuristic idea for selecting the next samples to train on for training deep learning models. This method relies on looking at the uncertainty of predictions of in the recent history of statements and preferring those instances that have a predictive uncertainty over the recent predictions. This allows the training method to train on instances that are neither too hard nor too easy and focus on reducing the uncertainty whenever it has the greatest potential gain to do so. There are two extra components that make this method work: - Windowing: only looking at the recent history of the instances which has two effects: firstly, the current state of the model is explored which gives a more recent assessment relative to the current state of the model. Secondly, it makes the algorithm faster by reducing the overhead of analyzing the prediction history of samples. - Annealing the selection bias: as the training goes on the selection becomes more random and less biased. This approach is evaluated in on three simple data-sets: MNIST, CIFAR-10 and CIFAR-100. Although this is a very limited subset of models, the results are consistent and statistically significant, although their effect is not really huge. The paper gives very little theoretical justification or analysis of the results but gives only the presented empirical evidence which seems to support the hypothesis on the efficacy of the approach. Another drawback of the approach is that it introduces new hyperparameters: those governing the annealing schedule for the selection bias. Since the approach seems efficient in a relatively constrained setup, it can be reasonably expected that it might be helpful in more general situations, therefore. On the other hand, since it is only evaluated on three very similar tasks, it limits the conclusiveness of the results. That*s why I would for weak accept. In the presence of more empirical (or even theoretical) evidence, I would vote for strong accept.
<|review3|>
Answer me with only the common concern that the reviewers have, with only the bullet points without any other explanation.
"""

answer = r"""
- Rigorous Experimentation: Many submissions fall short on experimental rigor—insufficient controls, lack of repeatability, vague protocol descriptions, or no statistical power analysis.
- Comparison to Prior Work: Authors often claim novelty without fair, head-to-head benchmarks against the most relevant baselines.
- Quantitative Claims Need Proof: Percentages are meaningless without a clear definition of the metric, baseline, and statistical significance. For example, "Our method is 20% better" is vague without context.
- Dataset Size & Diversity: Too-small or homogenous datasets lead to overfitting and findings that don't generalize.
"""

sheet = openpyxl.load_workbook("or_2020.xlsx").active

for row in range(5, 40, 3): # there should be sheet.max_row, but with my pc would take ~80 hours
    review1 = sheet.cell(row, 6).value
    review2 = sheet.cell(row+1, 6).value
    review3 = sheet.cell(row+2, 6).value
    final_question = get_main_question(review1, review2, review3)
    response = ollama.chat(model='qwen3:14b', messages = [
        {
            "role": "user",
            "content": question
        },
        {
            "role": "assistant",
            "content": answer
        },
        {
            "role": "user",
            "content": final_question
        }
    ])
    response = response['message']['content']
    if response.find('</think>') != -1:
        response = response.split('</think>')[1] # remove the think part
    final_result.write(response)
    final_result.seek(0, SEEK_END)

final_result.seek(0)
question = f"""
I have analized a dataset of OpenReview papers with reviews of the papers because I wanted to extract some common concern that the reviewers have, to help authors know in advance what are the common concerns. Now I have the big list of general concerns, can you help me creating a nice markdown with the concerns and best solutions? For example, this can be a possible result:
<|result|>
# 📝 Paper Writing Guidelines for Authors

This guide outlines key principles to help authors avoid common pitfalls and meet the expectations of peer reviewers and the broader research community.

## 1. 🧪 Rigorous Experimentation

Many submissions fall short on experimental rigor—insufficient controls, lack of repeatability, vague protocol descriptions, or no statistical power analysis.

**Best-Practice Solutions:**

1. **Define Hypotheses & Variables**

   * State null and alternative hypotheses explicitly.
   * Identify independent, dependent, and control variables.

2. **Use Proper Controls & Randomization**

   * When possible, include both positive and negative controls.
   * Randomly assign subjects or data batches to conditions to avoid bias.

3. **Replicate & Report Variability**

   * Run ≥ 3 independent trials; report means ± standard deviations.
   * If variation exceeds \~5%, investigate sources before concluding.

4. **Perform Power Analysis**

   * Calculate required sample size up-front to detect your expected effect size (e.g., Cohen's d = 0.4).
   * Document alpha (commonly 0.05) and desired power (commonly 0.8).

## 2. 📊 Comparison to Prior Work

Authors often claim novelty without fair, head-to-head benchmarks against the most relevant baselines.

**Best-Practice Solutions:**

1. **Conduct a Focused Literature Review**

   * Identify 3-5 seminal and recent papers tackling the same problem.
   * Highlight gaps your approach fills.

2. **Re-implement or Use Published Code**

   * Whenever possible, obtain or re-implement baseline methods rather than quoting old results.
   * Match preprocessing, hyperparameters, and evaluation metrics exactly.

3. **Present Results in Comparative Tables**

   ```markdown
   | Method        | Dataset     | Metric     | Result         |
   |---------------|-------------|------------|----------------|
   | Proposed      | X           | Accuracy   | 87.4 %         |
   | Baseline A    | X           | Accuracy   | 82.1 %         |
   | Baseline B    | X           | Accuracy   | 84.3 %         |
   ```

   * Include confidence intervals or standard errors for each entry.

4. **Discuss Trade-Offs**

   * If your method improves one metric but worsens another (e.g., speed vs. accuracy), analyze why.
   * Be transparent about limitations relative to baselines.

## 3. 🔍 Quantitative Claims Need Proof

“20 % improvement” sounds impressive—until readers realize there's no reported variance, p-value, or context for that number.

**Best-Practice Solutions:**

1. **State Claims Early & Precisely**

   * In the Abstract/Introduction, quantify your main gain (e.g., “Our model reduces error by 20 %.”).

2. **Prove Them in the Results Section**

   * Show tabular or graphical results with error bars (95 % CI) or p-values.
   * Example:

     ```markdown
     | Model         | Mean Error | 95 % CI      | p-value |
     |---------------|------------|--------------|---------|
     | Ours          | 0.12       | [0.10, 0.14] | 0.003   |
     | Baseline      | 0.15       | [0.13, 0.17] | —       |
     ```

3. **Use Appropriate Statistical Tests**

   * T-tests, ANOVA, or non-parametric tests depending on distribution.
   * Correct for multiple hypotheses when reporting several metrics.

4. **Explain Practical Impact**

   * Beyond percentages, describe what a 20 % reduction means in domain terms (e.g., 2 days saved in processing time).

## 4. 📉 Dataset Size & Diversity

Too-small or homogenous datasets lead to overfitting and findings that don't generalize.

**Best-Practice Solutions:**

1. **Follow “Rule of Ten” When Feasible**

   * Aim for ≥ 10 samples per feature dimension (e.g., 50 features → ≥ 500 samples).

2. **Leverage Power-Based Sample Size Calculations**

   * For a medium effect (d = 0.4), two groups need \~100 samples each.

3. **Document Dataset Splits & Diversity**

   * Report train/validation/test sizes and demographic or source diversity (e.g., multiple sites, time periods).

4. **Augment When Necessary**

   * If real-world data are limited, consider data augmentation, transfer learning, or synthetic data—always flag these in methods.

5. **Assess Generalizability**

    * Perform sensitivity analyses: how do results change if you drop 10-20 % of data or vary domain characteristics?
<|result|>
And these are the concerns:
<|concerns|>
{final_result.read()}
<|concerns|>
Answer me with only the final guideline markdown.
"""

response = ollama.chat(model='qwen3:14b', messages = [
    {
        "role": "user",
        "content": question
    }
])
response = response['message']['content']
print(response)


# 📝 Paper Writing Guidelines for Authors

This guide outlines key principles to help authors avoid common pitfalls and meet the expectations of peer reviewers and the broader research community.

## 1. 🧪 Rigorous Experimentation

Many submissions fall short on experimental rigor—insufficient controls, lack of repeatability, vague protocol descriptions, or no statistical power analysis.

**Best-Practice Solutions:**

1. **Define Hypotheses & Variables**

   * State null and alternative hypotheses explicitly.
   * Identify independent, dependent, and control variables.

2. **Use Proper Controls & Randomization**

   * When possible, include both positive and negative controls.
   * Randomly assign subjects or data batches to conditions to avoid bias.

3. **Replicate & Report Variability**

   * Run ≥ 3 independent trials; report means ± standard deviations.
   * If variation exceeds \~5%, investigate sources before concluding.

4. **Perform Power Analysis**

   * Calculate required samp

<>:7: SyntaxWarning: invalid escape sequence '\~'
<>:7: SyntaxWarning: invalid escape sequence '\~'
/var/folders/gj/gmsvyb2j4mb68dqn3c3mhq6m0000gn/T/ipykernel_7872/2667602092.py:7: SyntaxWarning: invalid escape sequence '\~'
  """


## 🔮 Future work

Additional work can be surely done, for example:

- **Clustering & Embeddings**
  Although I initially thought of TF‑IDF + K‑Means approach for grouping extracted concerns, I haven’t yet integrated it and I preferred to go all-in with LLMs.

- **Dynamic Context Windowing**  
  The problem of mitigating prompt‑length issues remains, I may consider retrieval‑augmented prompting: store past concerns in a lightweight vector database, since they grow fast in case we analyze all the papers.